## Lecture 9.4: Implementing DT from scratch

### Decision Trees

Decision Trees are popular **supervised machine learning algorithm** that can be used for both **classification** and **regression** tasks

The tree itself is a model in decision trees and we would like to estimate an **optimal tree structure** from the given training data.

- The internal nodes contains conditions on features. Depending on the outcome of the comparision, we take an appropriate path in the tree. The process is repeated until we reach a leaf note.
- In the case of classification, leaf nodes contain label and in case of regression, the prediction is obtained by taking sample mean of labels of the subset of training present in that leaf node.

In this colab, we will implement decision tree for classification with ID3 algorithm

#### Importing Libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
eps = np.finfo(float).eps
eps

2.220446049250313e-16

Here `eps` is the smallest respectable number. At times we get `log(0)` or `0` in the denominator, to avoid that we are going to use this.

#### Classification Demo
In this case we'll use a synthetic data for classification data.

In [3]:
df = pd.read_csv('data/weather_play.csv') # This is the data shown in the slides
df.head()

,Outlook,Temperature,Humidity,Wind,Play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [4]:
df.shape

(14, 5)

In [5]:
df.values

array([['Sunny', 'Hot', 'High', 'Weak', 'No'],
       ['Sunny', 'Hot', 'High', 'Strong', 'No'],
       ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
       ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
       ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
       ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
       ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
       ['Sunny', 'Mild', 'High', 'Weak', 'No'],
       ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
       ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
       ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
       ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
       ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
       ['Rain', 'Mild', 'High', 'Strong', 'No']], dtype=object)

In [6]:
df.keys()

Index(['Outlook', 'Temperature', 'Humidity', 'Wind', 'Play'], dtype='object')

In [7]:
target = df.keys()[-1] # Name of the target column
target

'Play'

In [8]:
df.keys()[:-1]

Index(['Outlook', 'Temperature', 'Humidity', 'Wind'], dtype='object')

Let's check the total number of labels

In [9]:
df[target].unique()

array(['No', 'Yes'], dtype=object)

There are two labels `Yes` and `No`

In [10]:
print(df[target].unique()[0])
print(df[target].unique()[1])

No
Yes


In [12]:
print(df[target].value_counts()[df[target].unique()[0]])
print(df[target].value_counts()[df[target].unique()[1]])

5
9


9 out of 14 examples have `Yes` and 5 out of 14 examples have label `No`

#### Calculating entropy of the whole dataset

In [13]:
def find_entropy_whole(df):
    # Last column in dataframe is the target variable.
    target = df.keys()[-1]

    # Initialization
    overall_entropy = 0

    # possible values of the target
    values_in_target = df[target].unique()

    for value in values_in_target:
        p = df[target].value_counts()[value]/len(df[target])
        overall_entropy += -p*np.log2(p)
    
    return overall_entropy

find_entropy_whole(df)

0.9402859586706311

#### Calculating entropy of an attribute

In [14]:
def find_entropy_of_attribute(df, attribute):

    # last column in dataframe is the target variable
    target = df.keys()[-1]
    
    values_in_target = df[target].unique()

    # This gives different features in that attribute (
    #   like  'hot', 'cold' in temperature)
    values_in_attribute = df[attribute].unique()

    # Initialize attribute's entropy
    entropy_attribute = 0


    for value_in_attribute in values_in_attribute:
        overall_entropy = 0
        for value_in_target in values_in_target:
            num = len(df[attribute][df[attribute] == value_in_attribute][df[target] == value_in_target])
            den = len(df[attribute][df[attribute] == value_in_attribute])
            p = num/(den + eps)
            overall_entropy += -p*np.log2(p+eps)
        p2 = den/len(df)
        entropy_attribute += -p2*overall_entropy
    return abs(entropy_attribute)

Let's compute entropy for different attributes

In [15]:
for i_attribute in df.keys()[:-1]:
    print(f'Entropy of attribute \'{i_attribute}\' is :',
            find_entropy_of_attribute(df, i_attribute))

Entropy of attribute 'Outlook' is : 0.6935361388961914
Entropy of attribute 'Temperature' is : 0.9110633930116756
Entropy of attribute 'Humidity' is : 0.7884504573082889
Entropy of attribute 'Wind' is : 0.892158928262361


#### Finding the best attribute

In [16]:
def find_best_attribute_to_divide(df):
    # Information gain
    IG = []

    # All column names
    all_attribute_names = df.keys()[:-1]

    for attribute in all_attribute_names:
        # Compute information gain for every attribute
        IG.append(find_entropy_whole(df) - find_entropy_of_attribute(df, attribute))

    # Get the index of attribute with best information gain
    index_of_attribute_with_max_IG = np.argmax(IG)
    best_attribute = all_attribute_names[index_of_attribute_with_max_IG]

    return best_attribute

find_best_attribute_to_divide(df) 

'Outlook'

#### Building Decision Tree

In [19]:
def buildTree(df, tree=None):

    # last column in dataframe
    target = df.keys()[-1]

    # Here we build our decision tree

    # Get attribute with maximum information gain
    node = find_best_attribute_to_divide(df)

    # Get distinct value of that attribute
    attValue = np.unique(df[node])

    # Create an array dictionary to create tree
    if tree is None:
        tree = {}
        tree[node] = {}
    
    # We make a loop to contruct a tree by calling this function recursively
    # In this we check if the subset is pure and stops if it is pure
    for value in attValue:

        subtable = df[df[node] == value].reset_index(drop=True)
        clValue, counts = np.unique(subtable['Play'], return_counts=True)

        if len(counts) == 1: # Checking purity of the subset
            tree[node][value] = clValue[0]
        else:
            tree[node][value] = buildTree(subtable) # Calling the function recursively
    
    return tree

In [20]:
buildTree(df)

{'Outlook': {'Overcast': 'Yes',
  'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
  'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}

ID3 in its pure form performs no backtracking in its search. Once it selects an attribute to test at a particular level in the tree, it never backtracks to reconsider this choice. Therefore, it is susceptible to the usual risks of hill-climbing search without backtracking: converging to locally optimal solutions that are not globally optimal